Scrape KEGG for information to classify drugs from our dataset and visualize

In [2]:
import rnaseq_lib as r
import pandas as pd

from IPython.display import SVG

from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

## Generating Table

In [3]:
drugs = ['Abiraterone acetate', 'ADO-TRASTUZUMAB EMTANSINE', 'AFATINIB', 'ALECTINIB HYDROCHLORIDE', 'ALEMTUZUMAB', 
         'AXITINIB', 'BEVACIZUMAB', 'BICALUTAMIDE', 'BLINATUMOMAB', 'BOSUTINIB MONOHYDRATE', 'CABOZANTINIB', 
         'Capecitabine', 'CETUXIMAB', 'CRIZOTINIB', 'DABRAFENIB', 'DASATINIB', 'DOXORUBICIN HYDROCHLORIDE', 
         'Erlotinib', 'ERLOTINIB HYDROCHLORIDE', 'ETOPOSIDE', 'Everolimus', 'GEFITINIB', 'GEMTUZUMAB OZOGAMICIN', 
         'Goserelin acetate', 'IDELALISIB', 'IMATINIB MESYLATE', 'IPILIMUMAB', 'Ipilimumab', 'LAPATINIB',
         'LENALIDOMIDE', 'NECITUMUMAB', 'NILOTINIB', 'Nilotinib', 'NIVOLUMAB', 'OBINUTUZUMAB', 'OFATUMUMAB', 
         'OLAPARIB', 'PALBOCICLIB', 'PANITUMUMAB', 'PAZOPANIB HYDROCHLORIDE', 'PEMBROLIZUMAB', 'PERTUZUMAB', 
         'PONATINIB HYDROCHLORIDE', 'RAMUCIRUMAB', 'RITUXIMAB', 'SORAFENIB', 'Sorafenib', 'SUNITINIB MALATE', 
         'Sunitinib malate', 'TAMOXIFEN CITRATE', 'TEMSIROLIMUS', 'TRAMETINIB', 'VISMODEGIB', 'SORAFENIB', 
         'Sorafenib', 'SUNITINIB MALATE', 'Sunitinib malate', 'TAMOXIFEN CITRATE', 'TEMSIROLIMUS', 'TRAMETINIB', 
         'VISMODEGIB', 'ATEZOLIZUMAB', 'AVELUMAB', 'DURVALUMAB', 'Girentuximab', 'RUCAPARIB']

Use KEGG API to identify classification of drugs

In [ ]:
with open('drug-info.tsv', 'w') as f:
    f.write('Drug\tClass\tSubgroup\tSpecification\tName\n')
    for drug in drugs:
        row = [drug]
        drug_class = r.web.kegg.get_drug_class(drug)
        if drug_class:
            row.append(' '.join(drug_class[1].split()[1:]))
            row.append(' '.join(drug_class[2].split()[1:]))
            row.append(' '.join(drug_class[3].split()[1:]))
            row.append(' '.join(drug_class[4].split()[1:]))
            f.write('\t'.join(row) + '\n')
        else:
            print 'Nothing found for: {}'.format(drug)

Hand fix the 3-4 drugs that it couldn't classify and move output to **tables/drug-classification.tsv**

In [5]:
df = pd.read_csv('tables/drug-classification.tsv', sep='\t', index_col=0)
df.head(2)

,Class,Subgroup,Specification,Name
Drug,,,,
Abiraterone acetate,ENDOCRINE THERAPY,HORMONE ANTAGONISTS AND RELATED AGENTS,Other hormone antagonists and related agents,Abiraterone
ADO-TRASTUZUMAB EMTANSINE,ANTINEOPLASTIC AGENTS,OTHER ANTINEOPLASTIC AGENTS,Monoclonal antibodies,Trastuzumab emtansine


## Add Tissues to Dataframe

    Create empty dataframe
   
    For each drug:
        Collect drug info
        Grep for cancer terms
        Collect tissue corresponding to cancer
        For each tissue:
            Copy row and add tissue
            Add to new dataframe
        

In [6]:
# Store Series as rows
rows = []

for i, row in df.drop_duplicates().sort_values('Name').iterrows():
    drug = row.Name
    print drug
    
    # Get drug info
    info = r.web.kegg.get_drug_info(drug)
    
    # Grep for cancer terms
    candidates = set()
    for sentence in r.tissues.grep_cancer_terms(info):
        
        # Identify tissue from string
        new_candidates = r.tissues.identify_tissue_from_str(sentence)
        if new_candidates:
            print '\t', new_candidates, sentence
            candidates = candidates.union(new_candidates)
        
    # If tissue candidate, copy row and add column
    if candidates:
        for tissue in candidates:
            new_row = row.copy()
            new_row['Tissue'] = tissue
            rows.append(new_row)
        
    # No tissue found, add manually
    else:
        print '\tNo tissue found for: {}'.format(drug)
        i = raw_input('\tAdd tissue for drug or no to exit: ')
        while i != 'no':
            new_row = row.copy()
            new_row['Tissue'] = i
            rows.append(new_row)
            i = raw_input('\tAdd tissue for drug or no to exit: ')

Abiraterone
	set(['Prostate'])   DISEASE   Prostate cancer (castration-resistant) [DS:H00024]
Afatinib
	No tissue found for: Afatinib


KeyboardInterrupt: 

## Construct Dataframe and Process

In [ ]:
new = pd.concat(rows, axis=1).T
new = new.drop_duplicates()
new.head(2)

Drop drugs for Blood

In [ ]:
new = new[new.Tissue != 'Blood']

Simplify labels for plot

In [ ]:
class_rep = [('ENDOCRINE THERAPY', 'Endocrine'),
             ('ANTINEOPLASTIC AGENTS', 'Antineoplastic')]

sub_rep = [('HORMONE ANTAGONISTS AND RELATED AGENTS', 'Hormone Antagonists'),
            ('OTHER ANTINEOPLASTIC AGENTS', 'Other'),
            ('CYTOTOXIC ANTIBIOTICS AND RELATED SUBSTANCES', 'Cytotoxic Antibiotics'),
            ('PLANT ALKALOIDS AND OTHER NATURAL PRODUCTS', 'Plant Alkaloids'),
            ('HORMONES AND RELATED AGENTS', 'Hormones'),
            ('IMMUNOSUPPRESSANTS', 'Suppressants')]

spec_rep = [('Other hormone antagonists and related agents', 'Other antagonist'),
           ('Protein kinase inhibitors', 'Kinase inhibitors'),
           ('Anthracyclines and related substances', 'Anthracyclines'),
           ('Gonadotropin releasing hormone analogues', 'Gonadotropin analogue'),
           ('Other immunosuppressants', 'Other suppressant'),
           ('Other antineoplastic agents', 'Other agent')]

for entry, replacement in class_rep:
    new['Class'] = new.Class.replace(entry, replacement)

for entry, replacement in sub_rep:
    new['Subgroup'] = new.Subgroup.replace(entry, replacement)

for entry, replacement in spec_rep:
    new['Specification'] = new.Specification.replace(entry, replacement)
    
# Fix old categories
new['Tissue'] = new.Tissue.replace('Colon-Small_intestine', 'Colon')
new['Tissue'] = new.Tissue.replace('Skin-Head', 'Skin')

Fix formatting

In [9]:
new['Class'] = new.Class.apply(lambda x: x.lower().capitalize())
new['Subgroup'] = new.Subgroup.apply(lambda x: x.lower().capitalize())
new['Specification'] = new.Specification.apply(lambda x: x.lower().capitalize())

Save dataframe


In [11]:
new.to_csv('tables/drug-classification-tissue.tsv', sep='\t')

# Drug Classification Sankey Graph

Read in dataframe (if running again)

In [7]:
new = pd.read_csv('tables/drug-classification-tissue.tsv', sep='\t', index_col=0)

Create links automatically

In [8]:
def add_links(iteritems):
    links = []
    type_count = 0
    current_type = None

    for pair, count in iteritems:
        source, target = pair

        # Track type by grouping samples by "source"
        if source != current_type:
            current_type = source
            type_count += 1
        links.append({'source': source, 'target': target, 'value': count, 'type': type_count})
    return links

Create links between Class -> Subgroup -> Specification -> Tissue

In [9]:
links = []
links.extend(add_links(new.groupby('Class').Subgroup.value_counts().iteritems()))
links.extend(add_links(new.groupby('Subgroup').Specification.value_counts().iteritems()))
links.extend(add_links(new.groupby('Specification').Tissue.value_counts().iteritems()))

Define factory functon for Sankey plot

In [10]:
layout = Layout(width="1200", height="800")
def sankey(**value):
    """Show SankeyWidget with default values for size and margins"""
    return SankeyWidget(layout=layout,
                        margins=dict(top=50, bottom=0, left=180, right=150),
                        **value)

In [11]:
sankey(links=links)

SankeyWidget(layout=Layout(height=u'800', width=u'1200'), links=[{'source': 'Antineoplastic', 'type': 1, 'target': 'Other', 'value': 54}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Antimetabolites', 'value': 3}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Plant alkaloids', 'value': 2}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Cytotoxic antibiotics', 'value': 1}, {'source': 'Endocrine', 'type': 2, 'target': 'Hormone antagonists', 'value': 3}, {'source': 'Endocrine', 'type': 2, 'target': 'Hormones', 'value': 1}, {'source': 'Immunosuppressants', 'type': 3, 'target': 'Suppressants', 'value': 2}, {'source': 'Antimetabolites', 'type': 1, 'target': 'Pyrimidine analogues', 'value': 3}, {'source': 'Cytotoxic antibiotics', 'type': 2, 'target': 'Anthracyclines', 'value': 1}, {'source': 'Hormone antagonists', 'type': 3, 'target': 'Anti-androgens', 'value': 1}, {'source': 'Hormone antagonists', 'type': 3, 'target': 'Anti-estrogens', 'value': 1}, {'source': 'Hormone antagonists', 'type': 3, 'target': 'Other antagonist', 'value': 1}, {'source': 'Hormones', 'type': 4, 'target': 'Gonadotropin analogue', 'value': 1}, {'source': 'Other', 'type': 5, 'target': 'Monoclonal antibodies', 'value': 25}, {'source': 'Other', 'type': 5, 'target': 'Kinase inhibitors', 'value': 24}, {'source': 'Other', 'type': 5, 'target': 'Other agent', 'value': 4}, {'source': 'Other', 'type': 5, 'target': 'Parp inhibitor', 'value': 1}, {'source': 'Plant alkaloids', 'type': 6, 'target': 'Podophyllotoxin derivatives', 'value': 2}, {'source': 'Suppressants', 'type': 7, 'target': 'Other suppressant', 'value': 2}, {'source': 'Anthracyclines', 'type': 1, 'target': 'Ovary', 'value': 1}, {'source': 'Anti-androgens', 'type': 2, 'target': 'Prostate', 'value': 1}, {'source': 'Anti-estrogens', 'type': 3, 'target': 'Breast', 'value': 1}, {'source': 'Gonadotropin analogue', 'type': 4, 'target': 'Breast', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Kidney', 'value': 6}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Lung', 'value': 5}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Breast', 'value': 3}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Pancreas', 'value': 3}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Skin', 'value': 2}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Thyroid', 'value': 2}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Brain', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Liver', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Stomach', 'value': 1}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Colon', 'value': 5}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Lung', 'value': 5}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Bladder', 'value': 4}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Kidney', 'value': 3}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Breast', 'value': 2}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Skin', 'value': 2}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Stomach', 'value': 2}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Cervix', 'value': 1}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'Ovary', 'value': 1}, {'source': 'Other agent', 'type': 7, 'target': 'Breast', 'value': 1}, {'source': 'Other agent', 'type': 7, 'target': 'Ovary', 'value': 1}, {'source': 'Other agent', 'type': 7, 'target': 'Prostate', 'value': 1}, {'source': 'Other agent', 'type': 7, 'target': 'Skin', 'value': 1}, {'source': 'Other antagonist', 'type': 8, 'target': 'Prostate', 'value': 1}, {'source': 'Other suppressant', 'type': 9, 'target': 'Brain', 'value': 1}, {'source': 'Other suppressant', 'type': 9, 'target': 'Skin', 'value': 1}, {'source': 'Parp inhibitor', 'type': 10, 'target': 'Ovary', 'value': 1}, {'source': 'Podophyllotoxin derivatives', 'type': 11, 'target': 'Lung', 'value': 1}, {'source': 'Podophyllot

# Mechanism of Action 

Use hand-curated labeling to group drugs by _mechanism of action_

In [12]:
df = pd.read_csv('tables/drug-biomarkers.tsv', sep='\t', index_col=0)

Standardize index names

In [13]:
df.index = [x.lower().capitalize() for x in df.index]
new.index = [x.lower().capitalize() for x in new.index]

Set mechanism of action in new dataframe

In [14]:
new['Mechanism_of_Action'] = df['Mechanism of Action']

Define links

In [17]:
links = []
links.extend(add_links(new.groupby('Class').Specification.value_counts().iteritems()))
links.extend(add_links(new.groupby('Specification')['Mechanism_of_Action'].value_counts().iteritems()))
links.extend(add_links(new.groupby('Mechanism_of_Action').Tissue.value_counts().iteritems()))

In [18]:
sankey(links=links)

SankeyWidget(layout=Layout(height=u'800', width=u'1200'), links=[{'source': 'Antineoplastic', 'type': 1, 'target': 'Monoclonal antibodies', 'value': 25}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Kinase inhibitors', 'value': 24}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Other agent', 'value': 4}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Pyrimidine analogues', 'value': 3}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Podophyllotoxin derivatives', 'value': 2}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Anthracyclines', 'value': 1}, {'source': 'Antineoplastic', 'type': 1, 'target': 'Parp inhibitor', 'value': 1}, {'source': 'Endocrine', 'type': 2, 'target': 'Anti-androgens', 'value': 1}, {'source': 'Endocrine', 'type': 2, 'target': 'Anti-estrogens', 'value': 1}, {'source': 'Endocrine', 'type': 2, 'target': 'Gonadotropin analogue', 'value': 1}, {'source': 'Endocrine', 'type': 2, 'target': 'Other antagonist', 'value': 1}, {'source': 'Immunosuppressants', 'type': 3, 'target': 'Other suppressant', 'value': 2}, {'source': 'Anthracyclines', 'type': 1, 'target': 'TOP2-inhibition', 'value': 1}, {'source': 'Anti-androgens', 'type': 2, 'target': 'AR-inhibition', 'value': 1}, {'source': 'Anti-estrogens', 'type': 3, 'target': 'ERA/B-inhibition', 'value': 1}, {'source': 'Gonadotropin analogue', 'type': 4, 'target': 'GnRH-promotion, LHRH-promotion', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'mTOR-inhibition', 'value': 5}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'ErbB1-inhibition', 'value': 3}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'c/b-RAF-inhibition, VEGFR2/3-inhibition, PDGFB-inhibition, c-RET-inhibition, CD117/135-inhibition', 'value': 3}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'ALK-inhibition', 'value': 2}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'VEGFR1/2/3-inhibition, PDGFA/B-inhibition, c-RET-inhibition, CD117/135-inhibition', 'value': 2}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'BRAF-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'Bcr-Abl-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'CDK4/6-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'ErbB1/2-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'ErbB1/2/4-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'MEK1/2-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'PDGFA/B-inhibition, VGFR1/2/3-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'VEGFR1/2/3-inhibition', 'value': 1}, {'source': 'Kinase inhibitors', 'type': 5, 'target': 'c-MET-inhibition, VEGFR1/2/3-inhibition', 'value': 1}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'VEGFR1-inhibition', 'value': 5}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'PD-1-inhibition', 'value': 4}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'PDL1-inhibition', 'value': 4}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'ErbB1-inhibition', 'value': 3}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'VEGFR2-inhibition', 'value': 3}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'ErbB2-inhibition', 'value': 2}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'PD1-inhibition', 'value': 2}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'CAIX-inhibition', 'value': 1}, {'source': 'Monoclonal antibodies', 'type': 6, 'target': 'CD152-inhibition', 'value': 1}, {'source': 'Other agent', 'type': 7, 'target': 'PARP1-inhibition', 'value': 3}, {'source': 'Other agent', 'type': 7, 'target': 'SMO-inhibition', 'value': 1}, {'source': 'Other antagonist', 'type': 8, 'target': 'CYP17-inhition', 'value': 1}, {'source': 'Other suppressant', 'type': 9, 'target': 'TNF\xce\xb1-inhibition', 'value': 2}, {'source': 'Parp inhibitor', 'type': 10, '

# Biomarker / Mechanism of Action Sankey Graph

In [149]:
new['Biomarker'] = df['Biomarker(s) for Sankey Graph']

In [161]:
links = []
links.extend(add_links(new.groupby('Mechanism_of_Action').Biomarker.value_counts().iteritems()))
#links.extend(add_links(new.groupby('Biomarker').Mechanism_of_Action.value_counts().iteritems()))

In [162]:
layout = Layout(width="1200", height="800")
def sankey(**value):
    """Show SankeyWidget with default values for size and margins"""
    return SankeyWidget(layout=layout,
                        margins=dict(top=50, bottom=0, left=300, right=300),
                        **value)

In [163]:
sankey(links=links)

SankeyWidget(layout=Layout(height=u'800', width=u'1200'), links=[{'source': 'ALK-inhibition', 'type': 1, 'target': 'ALK fusion', 'value': 1}, {'source': 'ALK-inhibition', 'type': 1, 'target': 'ALK fusion; ROS1 fusion', 'value': 1}, {'source': 'AR-inhibition', 'type': 2, 'target': 'AR overexpression', 'value': 1}, {'source': 'BRAF-inhibition', 'type': 3, 'target': 'BRAF (V600E/K); G6PD (S218F, V98M, N156D), biallelic inactivation [must not be deficient]', 'value': 1}, {'source': 'Bcr-Abl-inhibition', 'type': 4, 'target': 'BCR-ABL1 fusion; FIP1L1-PDGFRA fusion; COL1A1-PDGFRB fusion; PDGFRA (552-596, 631-668, 814-854), inframe deletion (I843);  KIT mutation in exon 9,11,13,14, 17', 'value': 1}, {'source': 'CAIX-inhibition', 'type': 5, 'target': 'CAIX overexpression', 'value': 1}, {'source': 'CD152-inhibition', 'type': 6, 'target': 'High serum VEGF, AEC, Tregs, and RLC levels + low LDH, AMC, and MDSCs', 'value': 1}, {'source': 'CDK4/6-inhibition', 'type': 7, 'target': 'ERBB2 expression; ESR1 overexpression', 'value': 1}, {'source': 'CYP17-inhition', 'type': 8, 'target': 'AR (L702H,T878A), amplification; CYP17 expression', 'value': 1}, {'source': 'ERA/B-inhibition', 'type': 9, 'target': 'ERBB2 expression; ESR1 overexpression', 'value': 1}, {'source': 'ErbB1-inhibition', 'type': 10, 'target': 'EGFR (L858R,L861,G719,S768I), exon 19 deletions, activating mutations within exons 18\xe2\x80\x9321 of TKI domain', 'value': 2}, {'source': 'ErbB1-inhibition', 'type': 10, 'target': 'EGFR (L858R), exon 19 deletions, amplification', 'value': 1}, {'source': 'ErbB1-inhibition', 'type': 10, 'target': 'EGFR overexpression', 'value': 1}, {'source': 'ErbB1-inhibition', 'type': 10, 'target': 'EGFR overexpression; KRAS (G13D)', 'value': 1}, {'source': 'ErbB1-inhibition', 'type': 10, 'target': 'Wild-type KRAS; wild-type NRAS', 'value': 1}, {'source': 'ErbB1/2-inhibition', 'type': 11, 'target': 'ERBB2 overexpression, amplification', 'value': 1}, {'source': 'ErbB1/2/4-inhibition', 'type': 12, 'target': 'EGFR (L858R, L861P/Q/R, G719A/C/D/S, L747S, S768I), insertions/deletions (exon 19)', 'value': 1}, {'source': 'ErbB2-inhibition', 'type': 13, 'target': 'ERBB2 amplificaition, overexpression', 'value': 1}, {'source': 'ErbB2-inhibition', 'type': 13, 'target': 'ERBB2 overexpression, amplification', 'value': 1}, {'source': 'GnRH-promotion, LHRH-promotion', 'type': 14, 'target': 'GNRHR1 overexpression', 'value': 1}, {'source': 'MEK1/2-inhibition', 'type': 15, 'target': 'BRAF (V600E/K)', 'value': 1}, {'source': 'PARP1-inhibition', 'type': 16, 'target': 'BRCA1 oncogenic mutation; BRCA2 oncogenic mutation', 'value': 4}, {'source': 'PD-1-inhibition', 'type': 17, 'target': 'CD274 overexpression', 'value': 4}, {'source': 'PD1-inhibition', 'type': 18, 'target': 'CD274 overexpression', 'value': 2}, {'source': 'PDGFA/B-inhibition, VGFR1/2/3-inhibition', 'type': 19, 'target': 'UGT1A1 biallelic inactivation', 'value': 1}, {'source': 'PDL1-inhibition', 'type': 20, 'target': 'CD274 overexpression', 'value': 4}, {'source': 'SMO-inhibition', 'type': 21, 'target': 'PTCH1 oncogenic mutation [loss of function]', 'value': 1}, {'source': 'TNF\xce\xb1-inhibition', 'type': 22, 'target': 'MDS: 5q- syndrome (3 adjacent genes deleted: CSF2, PDGFRB, and CSF1R)', 'value': 2}, {'source': 'TOP2-inhibition', 'type': 23, 'target': '???', 'value': 2}, {'source': 'TOP2-inhibition', 'type': 23, 'target': 'TP53 oncogenic mutation', 'value': 1}, {'source': 'Thymidylate synthase-inhibition', 'type': 24, 'target': 'DPYD (I560S,D949V), splice donor variant (DPYD*2A)', 'value': 3}, {'source': 'VEGFR1-inhibition', 'type': 25, 'target': 'VHL inactivation (by mutation or promoter hypermethylation)', 'value': 5}, {'source': 'VEGFR1/2/3-inhibition', 'type': 26, 'target': 'ABL1 (T315I); VHL inactivation (by mutation or promoter hypermethylation)', 'value': 1}, {'source': 'VEGFR1/2/3-inhibition, PDGFA/B-inhibition, c-RET-inhibition, CD117/135-inhibition', 'type': 27, 'target': 'PDGFRA (552-596,631-668,814-85